In [182]:
import sys, re, time, string
import numpy as n
from scipy.special import gammaln, psi

n.random.seed(100000001)
meanchangethresh = 0.000001

def dirichlet_expectation(alpha):
    """
    For a vector theta ~ Dir(alpha), computes E[log(theta)] given alpha.
    """
    if (len(alpha.shape) == 1):
        return(psi(alpha) - psi(n.sum(alpha)))
    return(psi(alpha) - psi(n.sum(alpha, 1))[:, n.newaxis])

def parse_doc_list(docs, vocab):
    """
    Parse a document into a list of word ids and a list of counts,
    or parse a set of documents into two lists of lists of word ids
    and counts.

    Arguments: 
    docs:  List of D documents. Each document must be represented as
           a single string. (Word order is unimportant.) Any
           words not in the vocabulary will be ignored.
    vocab: Dictionary mapping from words to integer ids.

    Returns a pair of lists of lists. 

    The first, wordids, says what vocabulary tokens are present in
    each document. wordids[i][j] gives the jth unique token present in
    document i. (Don't count on these tokens being in any particular
    order.)

    The second, wordcts, says how many times each vocabulary token is
    present. wordcts[i][j] is the number of times that the token given
    by wordids[i][j] appears in document i.
    """
    if (type(docs).__name__ == 'str'):
        temp = list()
        temp.append(docs)
        docs = temp

    D = len(docs)
    
    wordids = list()
    wordcts = list()
    for d in range(0, D):
        #docs[d] = str(docs[d].encode('raw_unicode_escape')).lower()
        #docs[d] = re.sub(r'-', ' ', docs[d])
        #docs[d] = re.sub(r'[^a-z ]', '', docs[d])
        #docs[d] = re.sub(r' +', ' ', docs[d])
        words = string.split(docs[d])
        ddict = dict()
        for word in words:
            if (word in vocab):
                wordtoken = vocab[word]
                if (not wordtoken in ddict):
                    ddict[wordtoken] = 0
                ddict[wordtoken] += 1
        wordids.append(ddict.keys())
        wordcts.append(ddict.values())

    return((wordids, wordcts))

class OnlineLDA:
    """
    Implements online VB for LDA as described in (Hoffman et al. 2010).
    """

    def __init__(self, vocab, K, D, alpha, eta, tau0, kappa):
        """
        Arguments:
        K: Number of topics
        vocab: A set of words to recognize. When analyzing documents, any word
           not in this set will be ignored.
        D: Total number of documents in the population. For a fixed corpus,
           this is the size of the corpus. In the truly online setting, this
           can be an estimate of the maximum number of documents that
           could ever be seen.
        alpha: Hyperparameter for prior on weight vectors theta
        eta: Hyperparameter for prior on topics beta
        tau0: A (positive) learning parameter that downweights early iterations
        kappa: Learning rate: exponential decay rate---should be between
             (0.5, 1.0] to guarantee asymptotic convergence.

        Note that if you pass the same set of D documents in every time and
        set kappa=0 this class can also be used to do batch VB.
        """
        self._vocab = dict()
        for word in vocab:
            word = word.lower()
            word = re.sub(r'[^a-z]', '', word)
            self._vocab[word] = len(self._vocab)

        self._K = K
        self._W = len(self._vocab)
        self._D = D
        self._alpha = alpha
        self._eta = eta
        self._tau0 = tau0 + 1
        self._kappa = kappa
        self._updatect = 0

        # Initialize the variational distribution q(beta|lambda)
        self._lambda = 1*n.random.gamma(100., 1./100., (self._K, self._W))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)

    def do_e_step(self, docs):
        """
        Given a mini-batch of documents, estimates the parameters
        gamma controlling the variational distribution over the topic
        weights for each document in the mini-batch.

        Arguments:
        docs:  List of D documents. Each document must be represented
               as a string. (Word order is unimportant.) Any
               words not in the vocabulary will be ignored.

        Returns a tuple containing the estimated values of gamma,
        as well as sufficient statistics needed to update lambda.
        """
        # This is to handle the case where someone just hands us a single
        # document, not in a list.
        if (type(docs).__name__ == 'string'):
            temp = list()
            temp.append(docs)
            docs = temp

        (wordids, wordcts) = parse_doc_list(docs, self._vocab)
        batchD = len(docs)

        # Initialize the variational distribution q(theta|gamma) for
        # the mini-batch
        gamma = 1*n.random.gamma(100., 1./100., (batchD, self._K))
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = n.exp(Elogtheta)

        sstats = n.zeros(self._lambda.shape)
        # Now, for each document d update that document's gamma and phi
        it = 0
        meanchange = 0
        for d in range(0, batchD):
            # These are mostly just shorthand (but might help cache locality)
            ids = wordids[d]
            cts = wordcts[d]
            gammad = gamma[d, :]
            Elogthetad = Elogtheta[d, :]
            expElogthetad = expElogtheta[d, :]
            expElogbetad = self._expElogbeta[:, ids]
            # The optimal phi_{dwk} is proportional to 
            # expElogthetad_k * expElogbetad_w. phinorm is the normalizer.
            phinorm = n.dot(expElogthetad, expElogbetad) + 1e-100
            # Iterate between gamma and phi until convergence
            for it in range(0, 100):
                lastgamma = gammad
                # We represent phi implicitly to save memory and time.
                # Substituting the value of the optimal phi back into
                # the update for gamma gives this update. Cf. Lee&Seung 2001.
                gammad = self._alpha + expElogthetad * \
                    n.dot(cts / phinorm, expElogbetad.T)
                Elogthetad = dirichlet_expectation(gammad)
                expElogthetad = n.exp(Elogthetad)
                phinorm = n.dot(expElogthetad, expElogbetad) + 1e-100
                # If gamma hasn't changed much, we're done.
                meanchange = n.mean(abs(gammad - lastgamma))
                if (meanchange < meanchangethresh):
                    break
            gamma[d, :] = gammad
            # Contribution of document d to the expected sufficient
            # statistics for the M step.
            sstats[:, ids] += n.outer(expElogthetad.T, cts/phinorm)

        # This step finishes computing the sufficient statistics for the
        # M step, so that
        # sstats[k, w] = \sum_d n_{dw} * phi_{dwk} 
        # = \sum_d n_{dw} * exp{Elogtheta_{dk} + Elogbeta_{kw}} / phinorm_{dw}.
        sstats = sstats * self._expElogbeta

        return((gamma, sstats))

    def update_lambda(self, docs):
        """
        First does an E step on the mini-batch given in wordids and
        wordcts, then uses the result of that E step to update the
        variational parameter matrix lambda.

        Arguments:
        docs:  List of D documents. Each document must be represented
               as a string. (Word order is unimportant.) Any
               words not in the vocabulary will be ignored.

        Returns gamma, the parameters to the variational distribution
        over the topic weights theta for the documents analyzed in this
        update.

        Also returns an estimate of the variational bound for the
        entire corpus for the OLD setting of lambda based on the
        documents passed in. This can be used as a (possibly very
        noisy) estimate of held-out likelihood.
        """

        # rhot will be between 0 and 1, and says how much to weight
        # the information we got from this mini-batch.
        rhot = pow(self._tau0 + self._updatect, -self._kappa)
        self._rhot = rhot
        # Do an E step to update gamma, phi | lambda for this
        # mini-batch. This also returns the information about phi that
        # we need to update lambda.
        (gamma, sstats) = self.do_e_step(docs)
        # Estimate held-out likelihood for current values of lambda.
        #bound = self.approx_bound(docs, gamma)
        # Update lambda based on documents.
        self._lambda = self._lambda * (1-rhot) + \
            rhot * (self._eta + self._D * sstats / len(docs))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)
        self._updatect += 1

        return gamma

    def approx_bound(self, docs, gamma):
        """
        Estimates the variational bound over *all documents* using only
        the documents passed in as "docs." gamma is the set of parameters
        to the variational distribution q(theta) corresponding to the
        set of documents passed in.

        The output of this function is going to be noisy, but can be
        useful for assessing convergence.
        """

        # This is to handle the case where someone just hands us a single
        # document, not in a list.
        if (type(docs).__name__ == 'string'):
            temp = list()
            temp.append(docs)
            docs = temp

        (wordids, wordcts) = parse_doc_list(docs, self._vocab)
        batchD = len(docs)

        score = 0
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = n.exp(Elogtheta)

        # E[log p(docs | theta, beta)]
        for d in range(0, batchD):
            gammad = gamma[d, :]
            ids = wordids[d]
            cts = n.array(wordcts[d])
            phinorm = n.zeros(len(ids))
            for i in range(0, len(ids)):
                temp = Elogtheta[d, :] + self._Elogbeta[:, ids[i]]
                tmax = max(temp)
                phinorm[i] = n.log(sum(n.exp(temp - tmax))) + tmax
            score += n.sum(cts * phinorm)
#             oldphinorm = phinorm
#             phinorm = n.dot(expElogtheta[d, :], self._expElogbeta[:, ids])
#             print oldphinorm
#             print n.log(phinorm)
#             score += n.sum(cts * n.log(phinorm))

        # E[log p(theta | alpha) - log q(theta | gamma)]
        score += n.sum((self._alpha - gamma)*Elogtheta)
        score += n.sum(gammaln(gamma) - gammaln(self._alpha))
        score += sum(gammaln(self._alpha*self._K) - gammaln(n.sum(gamma, 1)))

        # Compensate for the subsampling of the population of documents
        score = score * self._D / len(docs)

        # E[log p(beta | eta) - log q (beta | lambda)]
        score = score + n.sum((self._eta-self._lambda)*self._Elogbeta)
        score = score + n.sum(gammaln(self._lambda) - gammaln(self._eta))
        score = score + n.sum(gammaln(self._eta*self._W) - 
                              gammaln(n.sum(self._lambda, 1)))

        return(score)


In [103]:
# The number of documents to analyze each iteration
batchsize = 50
# The total number of documents in Wikipedia
D = 3800
# The number of topics
K = 40

In [154]:
vocab =n.load('temp/Lasvegas_data/lv_vocab10.npy')

In [157]:
vocab = list(vocab)

In [158]:
toremove = ['food','place','service','time','restaurant','menu','order','price','dish','table','thing','person','experience','server','staff','way','bit','portion','lot','hour','star','plate','course','option','customer','liking']

In [159]:
for w in toremove:
    vocab.remove(w)

In [160]:
W = len(vocab)

In [161]:
W

10338

In [163]:
vocab  = dict(zip(vocab,range(len(vocab))))

In [165]:
olda = OnlineLDA(vocab, K, D, 1./K, 1./K, 1024., 0.7)

In [166]:
documentstoanalyze = int(D/batchsize)
documentstoanalyze

76

In [12]:
import json
with open('../Zipped_data_LV/lasvegas_word_adjective.json', 'r') as fp:
    lv_wadj = json.load(fp)

In [171]:
docsetbig = [] 
articlenamesbig = []

for k in lv_wadj.keys()[:3800]:
    docsetbig.append(" ".join(lv_wadj[k][0][0]))
    articlenamesbig.append(k)

In [172]:
docsetbig = n.split(n.array(docsetbig),76)

In [183]:
for iteration in range(0, documentstoanalyze):
    # Download some articles
    docset = docsetbig[iteration]
    # Give them to online LDA
    gamma = olda.update_lambda(docset)        
    # Compute an estimate of held-out perplexity
    #(wordids, wordcts) = parse_doc_list(docset, olda._vocab)
    print '%d:  rho_t = %f,  held-out perplexity estimate = %f' % (iteration, olda._rhot, n.exp(-perwordbound))

0:  rho_t = 0.007807,  held-out perplexity estimate = 1007.972812
1:  rho_t = 0.007802,  held-out perplexity estimate = 1007.972812
2:  rho_t = 0.007797,  held-out perplexity estimate = 1007.972812
3:  rho_t = 0.007791,  held-out perplexity estimate = 1007.972812
4:  rho_t = 0.007786,  held-out perplexity estimate = 1007.972812
5:  rho_t = 0.007781,  held-out perplexity estimate = 1007.972812
6:  rho_t = 0.007775,  held-out perplexity estimate = 1007.972812
7:  rho_t = 0.007770,  held-out perplexity estimate = 1007.972812
8:  rho_t = 0.007765,  held-out perplexity estimate = 1007.972812
9:  rho_t = 0.007760,  held-out perplexity estimate = 1007.972812
10:  rho_t = 0.007754,  held-out perplexity estimate = 1007.972812
11:  rho_t = 0.007749,  held-out perplexity estimate = 1007.972812
12:  rho_t = 0.007744,  held-out perplexity estimate = 1007.972812
13:  rho_t = 0.007739,  held-out perplexity estimate = 1007.972812
14:  rho_t = 0.007733,  held-out perplexity estimate = 1007.972812
15:  

In [109]:
olda._lambda.shape

(40, 10235)

In [186]:
vocab['document']

3653

In [188]:
topic_word = olda._lambda  # model.components_ also works
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    topic_words = n.array(vocab.keys())[n.argsort(topic_dist)][:-(n_top_words+1):-1]
    print(u'Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: campechana buttery roll rice kebab mention flow musubis bummer abalone
Topic 1: fundido shumai cristo sidekick test solace cetera prick motherland sud
Topic 2: motherland horde brioch outbreak par sheep crunchy drunkenness bummer flick
Topic 3: sud motherland fondant carry-out band offender par prick mention wine
Topic 4: motherland outbreak brioch horde bummer crunchy discontent retreat par mention
Topic 5: leeway yelpster clove visit lassi artichoke gyozas generation coating mention
Topic 6: carry-out motherland outbreak horde taylor sheep spectacle spot brioch prick
Topic 7: outbreak carry-out folk frittus mention motherland discontent prick eavesdropping fork
Topic 8: motherland outbreak bummer horde par sheep bacon brioch discontent crunchy
Topic 9: motherland horde prick brioch wine bacon fiasco sheep abalone outbreak
Topic 10: horde brioch motherland bummer chap outbreak retreat crunchy fiasco jowl
Topic 11: mention pinball outbreak motherland prick fork coating bummer 

In [76]:
max(olda._lambda[0]

7.6462190424550149

In [82]:
testlambda = olda._lambda
for k in range(0, len(testlambda)):
    lambdak = list(testlambda[k, :])
    lambdak = lambdak / sum(lambdak)        
    temp = zip(lambdak, range(0, len(lambdak)))
    temp = sorted(temp, key = lambda x: x[0], reverse=True)
    print 'topic %d:' % (k)
    # feel free to change the "53" here to whatever fits your screen nicely.
    for i in range(0, 53):
        print '%20s  \t---\t  %.4f' % (vocab[temp[i][1]], temp[i][0])
    print

topic 0:
             verdict  	---	  0.0013
            chipotle  	---	  0.0013
               creme  	---	  0.0010
              liquid  	---	  0.0008
                bias  	---	  0.0006
                  ox  	---	  0.0004
             service  	---	  0.0004
                time  	---	  0.0004
                 par  	---	  0.0003
           childhood  	---	  0.0003
              kickin  	---	  0.0003
          spicy-ness  	---	  0.0003
                comp  	---	  0.0003
            estrella  	---	  0.0003
            peruvian  	---	  0.0003
             gelatus  	---	  0.0003
              bummer  	---	  0.0003
               green  	---	  0.0003
               stare  	---	  0.0002
             peasant  	---	  0.0002
              person  	---	  0.0002
                rice  	---	  0.0002
               diego  	---	  0.0002
            brandade  	---	  0.0002
                vest  	---	  0.0002
              hookah  	---	  0.0002
           seasoning  	---	  0.0002
                yol

In [91]:
gamma.shape

(50, 70)

In [ ]:
def kl_div_sym(v1,v2):
    n.sum(v*)